In [1]:
import random
T = 30
score = [random.randrange(2, 20, 2) for i in range(T)]
cost = [random.randrange(50, 100, 10) for i in range(T)]
area = [random.randrange(5, 15, 2) for i in range(T)]
C = 5000
A = 250
N = 200    # no. of people

In [2]:
print(score, cost, area, sep='\n')

[4, 12, 12, 10, 2, 4, 8, 14, 10, 8, 16, 4, 6, 14, 6, 4, 12, 14, 12, 8, 4, 14, 6, 8, 12, 4, 10, 6, 16, 10]
[90, 90, 90, 80, 60, 60, 80, 60, 70, 90, 70, 50, 80, 80, 60, 80, 50, 90, 90, 50, 50, 80, 50, 80, 90, 50, 50, 60, 60, 80]
[13, 9, 7, 5, 5, 9, 13, 13, 7, 5, 9, 5, 9, 5, 11, 7, 11, 5, 9, 11, 13, 11, 11, 13, 5, 13, 9, 5, 5, 5]


In [3]:
# DP knapsack to select based on each constraint i.e
# - on Area A
# - on Cost C

def dp_select(W1, W2, n, val, wts1, wts2, sel):
    if n == 0 or W1 == 0 or W2 == 0:
        return 0
    if wts1[n-1] > W1 or wts2[n-1] > W2:
        return dp_select(W1, W2, n-1, val, wts1, wts2, sel)
    sel1 = sel[:]
    sel2 = sel[:]
    v1 = dp_select(W1, W2, n-1, val, wts1, wts2, sel1)
    v2 = val[n-1] + dp_select(W1 - wts1[n-1], W2 - wts2[n-1], n, val, wts1, wts2, sel2)
    if v1 <= v2:
        sel[:] = sel2[:]
        sel[n-1] += 1
        return v2
    sel[:] = sel1[:]
    return v1

In [4]:
# do not run this cell below ↓↓↓↓↓↓↓↓

sel = [0]*T
dp_select(C, A, T, score, cost, area, sel)

print(*score)
print(*cost)
print(*area)
print(*sel)

as the Dynamic Programming method for optimal solution is very slow, we will be using Genetic Algorithm based search for obtaining a non-optimal but feasible solution.

# Genetic Algorithm

In [5]:
# first create a sample set
def get_sample_set(area, cost, A, C, T):
    S = []
    minc, maxc = float('inf'), -float('inf')
    for i in range(T):
        count = min(C//cost[i], A//area[i])
        minc = min(count, minc)
        maxc = max(count, maxc)
        S.extend([i]*count)
    return S, minc, maxc

In [6]:
S, minc, maxc = get_sample_set(area, cost, A, C, T)
print(S, minc, maxc, sep='\n')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10

In [7]:
X = 20    # no. of chromosomes

In [8]:
def initialize_chromosomes(X, n, minc, maxc):
    chromosomes = [[0]*n for i in range(X)]
    for c in chromosomes:
        choose = random.randint(minc, maxc)
        idx = random.sample(range(len(c)), choose)
        for i in idx:
            c[i] = 1
    return chromosomes

In [19]:
chromosomes = initialize_chromosomes(X, len(S), minc, maxc)
for c in chromosomes:
    print(*c, sep='')

0000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000010000010000000000000000000000000010000000000000000000010000000000000000000100000000000000000000100000000000000000000000000000000100010000000000000000000000000000000010000000000000010000000010100000000000000100000000000000000000000000000000000000000000000000001000000010000000000000000000000000000000100000001000000010000000000010000000000000000000001000000001000000000000000000000010000000001000000000001000000000000000000000000000000000000000000000000000000101000000000000000000000000000100000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000010000000000000100000000001000000001000011000000000000000100000000000000000000000000000000000001000000000000000000000000000000000100000100010000000000000000000000000000000000000000000000000000000000000001000000000000010001000000000000000000000000000000000001000000001000000000000000000010000
10100000

0001000000000000000000000010000000100000000000000000000000000000000000000000000000000000000010000000000100000000000000001000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000100001000000100000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000010000000000000000100000000000000000000000000000000000010000000000000001000000000001000000000000000000000000000100000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000100010000000000000000000000000010000000000000000000000001100000001001000010000000000000000000000000000000000000000000000000000000100000000101000000000000000000000000010000000000100000000000000000000100000100000000000000000000000000000100000000000001000000000001000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000
01000000

# objective function
F = w1 * Σscore(i) / N  +  w2 * (Σarea(i) - A)  +  w3 * (Σcost(i) - C)


w1, w2, w3 are weights

N = population

A = Area available

C = Cost Budget available

In [20]:
# objective funtion
def calculate_fitness(total_score, total_cost, total_area, C, A, N):
    f = 10*total_score/N + total_area - A + total_cost - C
    return f

In [21]:
def get_fitness(chromosome, score=score, S=S, cost=cost, area=area, C=C, A=A, N=N):
    total_cost = sum(cost[S[i]] for i in range(len(chromosome)) if chromosome[i])
    total_area = sum(area[S[i]] for i in range(len(chromosome)) if chromosome[i])
    if total_cost > C or total_area > A:
        return -float('inf')
    total_score = sum(score[S[i]] for i in range(len(chromosome)) if chromosome[i])
    total_fitness = calculate_fitness(total_score, total_cost, total_area, C, A, N)
    return total_fitness

In [22]:
tot_fit = []
for c in chromosomes:
    tot_fit.append(get_fitness(c))
    print(*c, "\t", tot_fit[-1], sep='')

0000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000010000010000000000000000000000000010000000000000000000010000000000000000000100000000000000000000100000000000000000000000000000000100010000000000000000000000000000000010000000000000010000000010100000000000000100000000000000000000000000000000000000000000000000001000000010000000000000000000000000000000100000001000000010000000000010000000000000000000001000000001000000000000000000000010000000001000000000001000000000000000000000000000000000000000000000000000000101000000000000000000000000000100000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000010000000000000100000000001000000001000011000000000000000100000000000000000000000000000000000001000000000000000000000000000000000100000100010000000000000000000000000000000000000000000000000000000000000001000000000000010001000000000000000000000000000000000001000000001000000000000000000010000	-inf
101

0001000000000000000000000010000000100000000000000000000000000000000000000000000000000000000010000000000100000000000000001000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000100001000000100000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000010000000000000000100000000000000000000000000000000000010000000000000001000000000001000000000000000000000000000100000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000100010000000000000000000000000010000000000000000000000001100000001001000010000000000000000000000000000000000000000000000000000000100000000101000000000000000000000000010000000000100000000000000000000100000100000000000000000000000000000100000000000001000000000001000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000	-inf
010

In [23]:
def perform_crossovers(chromosomes, tot_fit):
    M = len(chromosomes[0])    # length of a chromosome
    X = len(chromosomes)       # no. of chromosomes
    sorted_chrom = [i[0] for i in sorted(zip(chromosomes, tot_fit),
                                         key=lambda i:i[1], reverse=True)][:X//2]
    for i in range(X//2):
        pivot = random.randint(1, M-2)
        # array modified in place
        chromosomes[i] = sorted_chrom[i][:pivot] + sorted_chrom[X//2-i-1][pivot:]
        chromosomes[X-i-1] = sorted_chrom[X//2-i-1][:pivot] + sorted_chrom[i][pivot:]
        chromosomes[X//2-i-1] = sorted_chrom[i]
        chromosomes[X//2+i] = sorted_chrom[X//2-i-1]

In [24]:
perform_crossovers(chromosomes, tot_fit)

tot_fit = []
for c in chromosomes:
    tot_fit.append(get_fitness(c, score, S, cost, area, C, A))
    print(*c, "\t", tot_fit[-1], sep='')

0000000000000000000000000001000000000000000000000000101000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000100000000000000000000000000000000000000000000000000000000000000000001000000000000000000000001000100000000000000010000000100000000000000000000001000100000001000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000100000000000000000000100000000000000100000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000001000000110000000000010000000000000001000000000010000000100000000000100000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000001100010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000001000000000000000000000100000100000000000000000000100000000000000000000000000000000000000000010000000000000000000000000000000000000	-inf
110

0000000000000000000000000000000000100000000000000000000000000000000000000000100000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000001000001000001000000000000000000000000100000000000000000000000000000000100000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000001000000100000000000000000000000000000010000000000000000000100000001000000000000000000000000000000000000000000100000000000000000000000000001000000000000000000000000100000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000100100000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000001000000000001000000000000000010000000000000000000000100000000000000	-3150.2


In [25]:
def assign_fitness(chromosome, tot_fit, i):
    tot_fit[i] = get_fitness(chromosome)

In [26]:
from threading import Thread
from math import isclose

In [27]:
def run_genetic_algorithm(chromosomes, X, max_iter = 1000, max_rep = 20):
    rep_count = 0
    threads = [None]*X
    best_ch = None
    best_fit = -float('inf')
    tot_fit = [0]*X
    prev_fit = -float('inf')

    for t in range(max_iter):
        for i in range(X):
            threads[i] = Thread(target=assign_fitness, args=(chromosomes[i], tot_fit, i))
            threads[i].start()
        for th in threads:
            th.join()

        curr_best_fit, curr_best_ch = min(zip(tot_fit, chromosomes), key=lambda i:i[0])
        if curr_best_fit > best_fit:
            best_fit = curr_best_fit
            best_ch = curr_best_ch

        if t%50 == 0:
            print(curr_best_fit)

        if isclose(curr_best_fit, prev_fit, rel_tol=0.1):
            rep_count += 1
        prev_fit = curr_best_fit

        if rep_count >= max_rep:
            rep_count = 0 
            chromosomes = initialize_chromosomes(X, len(S), minc, maxc)
            continue
        perform_crossovers(chromosomes, tot_fit)
    
    return best_ch, best_fit

In [32]:
best_ch, best_fit = run_genetic_algorithm(chromosomes, X)
print(best_ch, best_fit)

-2515.1
-inf
-inf
-inf
-inf
-inf
-2411.7
-2246.2
-2214.5
-inf
-inf
-2022.1
-2293.9
-2240.7
-2092.2
-2477.7
-2235.0
-inf
-inf
-2466.5
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [33]:
from collections import defaultdict

In [34]:
def collect_idx_counts(chromosome, S):
    trees = defaultdict(int)
    for i in range(len(S)):
        if chromosome[i]:
            trees[S[i]] += 1
    return dict(trees)

In [35]:
trees = collect_idx_counts(best_ch, S)
trees

{1: 2,
 2: 1,
 3: 4,
 4: 2,
 5: 1,
 9: 2,
 10: 1,
 11: 3,
 13: 7,
 14: 1,
 17: 4,
 18: 2,
 19: 1,
 24: 3,
 27: 2,
 28: 2,
 29: 4}

In [36]:
tot_area = sum(area[i]*trees[i] for i in trees)
tot_cost = sum(cost[i]*trees[i] for i in trees)
tot_area, tot_cost

(248, 3210)

# things left to be done...
- fix the objective function...seems a bit odd, given the area utilisation and cost utilisation in above example; although might be due to random values of score, cost, area.
- in particular fix the weight w1, w2, w3.
_Note_ : negative objective function value might seem a bit odd but it is because of the constraints total_cost <= C and total_area <= A and hence, cost and area utilisation terms can have a maximum values of 0.

**Further...**:
- Apply this on the real data
- Incorporate with a front-end

 T- Tolerant, M – Moderate, IM- Intermediate, S-Sensitive, NA- Not Available, ET- Evergreen Tree, ES-Ever green Shrub, DT-Deciduous Tree
 
 Zone I- in and around the factory; Zone II- within a 2 km radius of the factory; Zone III- 2-4 km away from the factory; Zone IV- 4-6 km away and Zone V – a control zone 10-12 km away
from the industry

In [37]:
import pandas as pd

In [38]:
data = pd.read_csv("Tree-score.csv")
data

,Plant Species,Common name,Life form,Zone I,Zone II,Zone III,Zone IV,Canopy diameter (in m),Utility,Cost
0,Azadirachta indica,Neem,ET,T,T,T,T,10,5,3500
1,Albizia lebbeck,Siris,ET,T,T,T,T,8,3,4000
2,Bougainvillea spp.,Boganvilliea,ES,T,T,T,T,5,2,3000
3,Mangifera indica,Mango,ET,T,T,T,T,10,5,5000
4,Ficus bengalensis,Banyan,ET,T,T,T,T,15,5,4000
5,Murraya spp.,Murraya,ET,NaN,T,T,T,2,3,4000
6,Moringa oleifera,Moringa,DT,T,T,T,T,5,4,3500
7,Ficus religiosa,Peepal,ET,T,T,T,T,10,5,2000
8,Aagle marmelos,Bel,ET,M,T,T,T,5,4,3500
9,Psidium guajava,Guava,ES,M,T,T,T,5,4,4500


In [39]:
data = data.fillna('NA')
data['Area'] = data['Canopy diameter (in m)']**2

In [40]:
tree_data = []
fields = data.columns
for row in data.iterrows():
    item = dict(zip(fields, row[1]))
    tree_data.append(item)
tree_data

[{'Plant Species': 'Azadirachta indica',
  'Common name': 'Neem',
  'Life form': 'ET',
  'Zone I': 'T',
  'Zone II': 'T',
  'Zone III': 'T',
  'Zone IV': 'T',
  'Canopy diameter (in m)': 10,
  'Utility': 5,
  'Cost': 3500,
  'Area': 100},
 {'Plant Species': 'Albizia lebbeck',
  'Common name': 'Siris',
  'Life form': 'ET',
  'Zone I': 'T',
  'Zone II': 'T',
  'Zone III': 'T',
  'Zone IV': 'T',
  'Canopy diameter (in m)': 8,
  'Utility': 3,
  'Cost': 4000,
  'Area': 64},
 {'Plant Species': 'Bougainvillea spp.',
  'Common name': 'Boganvilliea',
  'Life form': 'ES',
  'Zone I': 'T',
  'Zone II': 'T',
  'Zone III': 'T',
  'Zone IV': 'T',
  'Canopy diameter (in m)': 5,
  'Utility': 2,
  'Cost': 3000,
  'Area': 25},
 {'Plant Species': 'Mangifera indica',
  'Common name': 'Mango',
  'Life form': 'ET',
  'Zone I': 'T',
  'Zone II': 'T',
  'Zone III': 'T',
  'Zone IV': 'T',
  'Canopy diameter (in m)': 10,
  'Utility': 5,
  'Cost': 5000,
  'Area': 100},
 {'Plant Species': 'Ficus bengalensis',
  '

In [41]:
def aqi_range(aqi):
    if aqi <= 50:
        return 'Good', 'Zone IV'
    elif 50 < aqi <= 100:
        return 'Moderate', 'Zone IV'
    elif 100 < aqi <= 150:
        return 'Unhealthy for sensitive groups', 'Zone III'
    elif 150 < aqi <= 200:
        return 'Unhealthy', 'Zone III'
    elif 200 < aqi <= 300:
        return 'Very Unhealthy', 'Zone II'
    else:
        return 'Hazardous', 'Zone I'

In [42]:
to_num = {'T':5, 'M':4, 'IM':3, 'S':2, 'NA':1}
for i in range(len(tree_data)):
    d = tree_data[i]
    d['Zone I'] = to_num[d['Zone I']]
    d['Zone II'] = to_num[d['Zone II']]
    d['Zone III'] = to_num[d['Zone III']]
    d['Zone IV'] = to_num[d['Zone IV']]
tree_data[0]

{'Plant Species': 'Azadirachta indica',
 'Common name': 'Neem',
 'Life form': 'ET',
 'Zone I': 5,
 'Zone II': 5,
 'Zone III': 5,
 'Zone IV': 5,
 'Canopy diameter (in m)': 10,
 'Utility': 5,
 'Cost': 3500,
 'Area': 100}

In [43]:
tree_idx = {tree_data[i]['Common name']:i for i in range(len(tree_data))}
print(tree_idx)

{'Neem': 0, 'Siris': 1, 'Boganvilliea': 2, 'Mango': 3, 'Banyan': 4, 'Murraya': 5, 'Moringa': 6, 'Peepal': 7, 'Bel': 8, 'Guava': 9, 'Tamarind': 10, 'Hibiscus Gurhal': 11, 'Dita bark': 12, 'Indian Elm': 13, 'False Ashoka': 14, 'Cordia': 15, 'Teak': 16, 'Eucalyptus': 17, 'Rose': 18, 'Palash': 19, 'Casuarina': 20, 'Mahogany': 21, 'Indian Rosewood': 22, 'Amla': 23, 'Ashok': 24, 'Sunflower': 25, 'Babool': 26}


In [44]:
def get_score(zone, tree_data=tree_data):
    scores = [0]*len(tree_data)
    for i in range(len(tree_data)):
        # f = w1*pi + w2*ui
        scores[i] = 10*tree_data[i][zone] + 5*tree_data[i]['Utility']
    return scores

In [47]:
zone = 'Zone III'
scores = get_score(zone)
print(*scores)

75 65 60 75 75 65 70 75 70 70 65 75 70 30 65 40 25 30 30 40 35 30 35 45 40 40 55


In [ ]:
X = 20
# chromosomes = initialize_chromosomes(X, )